### Given a csv file with "address", read it in and add add coordinates in "lat" and "lon" columns using Google Maps Geocoding API
https://developers.google.com/maps/documentation/geocoding/requests-geocoding

In [8]:
import json
import requests
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

In [2]:
# constants
key_json = json.load(open("credentials.json"))
gmaps_key = key_json["key"]
outputFormat = "json"
url = "https://maps.googleapis.com/maps/api/geocode/"

In [3]:
def get_lat_lon_for_address(row):
    address = row["street_and_house_no"] + ' ' + row["zipcode_and_city"]
    address_http = address.replace(' ', "%20").replace('+', "%2B")
    parameters = f"address={address_http}&key={gmaps_key}"
    request = f"{url}{outputFormat}?{parameters}"
    result = requests.get(request).json()
    location = result["results"][0]["geometry"]["location"]
    return pd.Series([location["lat"], location["lng"]])

In [4]:
df_scraped = pd.read_csv("../data/physicians.csv")
df = deepcopy(df_scraped)
tqdm.pandas()
df[["lat", "lon"]] = df.progress_apply(get_lat_lon_for_address, axis=1)
df.head()

,Unnamed: 0,name,expertise,distance,street_and_house_no,zipcode_and_city,telephone,lat,lon
0,0,Frau Dr. med. Christa Hutterer,"Fachärztin für Diagnostische Radiologie, Fachä...",0.1 km,Kaufingerstraße 15,80331 München-Altstadt-Lehel,089 / 2 00 01 43 - 50,48.137815,11.571804
1,1,Herr Dr. med. Josef J. Dohrenbusch,Facharzt für Allgemeinmedizin,0.1 km,Kaufingerstraße 12,80331 München,089 / 24 20 93 98,48.137615,11.573961
2,2,Frau Dr. med. Heidi Herrmann,Fachärztin für Innere Medizin,0.1 km,Altheimer Eck 2,80331 München,089 / 45 22 81 81,48.137541,11.571022
3,3,Herr Dr. med. (univ.) Thomas Wendel,Praktischer Arzt,0.1 km,Altheimer Eck 10,80331 München,089 / 89 67 40 20,48.137505,11.570328
4,4,Frau Dr. med. Sabine Konz,Praktische Ärztin,0.2 km,Eisenmannstraße 4,80331 München,089 / 37 02 97 67,48.137763,11.569028


In [5]:
df.to_csv("../data/physicians_with_coordinates.csv")

In [7]:
len(df)

1620